# Constucting a yeast cell from limited CryoET data
Tyler Earnest 2016/02/22

Rerun and Verified by Tianyu Wu. 2023/08/23
1. Remove the venv environment reqiurement.
2. Update the functions so that current script is compatible with LM v2.4.

Comments: 
+ you should first set up an anaconda environment with Lattice-Microbe v2.4 (Linux system required). So you can use the jLM packages for geometry manipulation. 
+ You can find and download the Lattice-Microbe from https://github.com/Luthey-Schulten-Lab/LM2.4_PNNL.


## 0.Imports

In [1]:
import itertools
import numpy as np
import scipy.spatial as spspat
import scipy.special as sps
import scipy.optimize as spopt
import scipy.interpolate as spinterp
import skimage.morphology as skmo
from functools import reduce
import pickle       
import lzma
try:
    %load_ext Cython 
    # This is for placement of ribosomes not include those in cryo-ET
except Exception as e:
    print(f"Cython loading error: {e}")
    print("Continuing without Cython - some functionality may be limited")

In [2]:
import matplotlib as mpl
import seaborn as sns

mplRc = {"axes.grid": False,
         "image.cmap":"inferno",
         "image.interpolation":"nearest",
         "image.origin":"lower"}

sns.set_context("talk", rc=mplRc)

for k,v in mplRc.items():
    mpl.rcParams[k] = v

import matplotlib.pyplot as plt
%matplotlib inline

### jLM imports

In [ ]:
# from venv import *
import os

jpcb_path = './data'  # change to the github repo
def dataFile(x):
    #return os.path.join(os.path.join(os.environ['VIRTUAL_ENV'], "data"), x)
    return os.path.join( jpcb_path, x)


from VolData import VolData
from jLM.RegionBuilder import RegionBuilder
# from jLM.RDME import Sim as RDMESim
# from jLM.RDME import File as RDMEFile
#from jLM.VmdUtil import tclTriangleMesh, tclTriangleMesh_scratch
#from jLM.JupyterDisplay import showBinaryLattices
import jLM

In [4]:
def report(name, value, fmt="{}"): 
    '''report function for print out information in format.'''
    label = "{:<25s}".format(name)
    value = "{:>25s}".format(fmt.format(value))
    print(label+value)

## Parameters

In [5]:
decimation = 18
cetPoreCount = 12 # pores in cet
cellVolume = 40 #  µm^3,  bnid:100430
cellWallFraction = 0.159
nucleusFraction = 0.07
envelopeThickness = 35e-9 #m
nucPoreRadius = 40e-9     #m
cellAspect = 1.5
nucleusAspect = 1.0
vacuoleFraction = 0.075
nMito = 3           #obtained from bionumbers
mitoFraction = 0.016
nRibosomes = 180000 #obtained from bionumbers

## 1.Load Cryo-ET data

In [ ]:
# voxels labeled with integers denoting type
labelData = VolData(dataFile("labels-2016_12_29_18_51_57")) 
cetLabels = dict(background=0,
                 nucEnv=1,
                 er=4,
                 mito=2,
                 membrane=3)

# LM lattice spacing defined by CET resolution
latticeSpacing = 1e-10*np.mean(labelData.dx)*decimation
labelOffset = [labelData.attr['origin_x'], 
               labelData.attr['origin_y'], 
               labelData.attr['origin_z']]

report("CET dimensions:",  labelData.scalar.shape)
report("CET spacing:",  1e-1*np.mean(labelData.dx), "{:.2f} nm")
report("LM spacing:",  latticeSpacing*1e9, "{:.2f} nm")

## Resample experimental data to LM lattice

In [7]:
def lmProc(im,threshold=0.01):
    """
    Process and downsample a 3D image to match Lattice Microbe's lattice requirements.
    
    This function:
    1. Transposes and downsamples the input image by a decimation factor
    2. Pads the output to ensure dimensions are multiples of 32
    3. Applies thresholding to create a binary output
    
    Args:
        im (ndarray): Input 3D image array with shape (z,y,x)
        threshold (float, optional): Threshold value for binarization. 
            Actual threshold is scaled by decimation^3. Defaults to 0.01.
    
    Returns:
        ndarray: Binary array where True indicates values above threshold after decimation.
            Output dimensions are padded to be multiples of 32.
            
    Notes:
        - Uses decimation factor defined in global scope
        - Transposes input from (z,y,x) to (y,x,z) order
        - Output shape is calculated as ceil(input_dim/(32*decimation))*32
        - Accumulates values from decimated points before thresholding
    """
    th = threshold*decimation**3
    src = im.transpose([1,2,0]).astype(np.int64)
    newShape = tuple(int(np.ceil(x/32/decimation)*32) 
                         for x in src.shape)
    deciShape = tuple(x//decimation for x in src.shape)
    dest = np.zeros(newShape, dtype=np.int64)
    for i,j,k in itertools.product(range(decimation),repeat=3):
        v = src[i::decimation, j::decimation, k::decimation]
        nx,ny,nz = v.shape
        dest[:nx,:ny,:nz] += v
    return dest>th

`return dest > th`: Finally, the function returns a boolean NumPy array where each element is True if it exceeds the threshold th, and False otherwise. This boolean array represents the result of the thresholding operation applied to the decimated and processed image.

In [8]:
cetNucEnvReg = lmProc(labelData.scalar==cetLabels['nucEnv'])
cetErReg = lmProc(labelData.scalar==cetLabels['er'])
cetMitoReg = lmProc(labelData.scalar==cetLabels['mito'])
cetMemReg = lmProc(labelData.scalar==cetLabels['membrane'])

## 2.LM lattice builder Initialization
Helper object to construct lattice
* Discrete geometric representation
* Constructive solid geometry on a lattice
* 3D binary morphology

In [ ]:
B = RegionBuilder(dims=cetErReg.shape)
report('cetERregionShape',cetErReg.shape)

## 3.Estimate the density of nuclear pores

In [ ]:
nucTmp = labelData.scalar==cetLabels['nucEnv']

flat = np.any(nucTmp,axis=2)
filled = skmo.closing(flat,skmo.disk(10))
skel = skmo.skeletonize(filled)

arclen = 1e-10*np.mean(labelData.dx)*np.sum(skel)
width = 1e-10*labelData.dx[1]*np.sum(np.any(nucTmp,axis=(0,1)))
sa = arclen*width
poreDensity = cetPoreCount/sa
report("Pore density", 1e-12*poreDensity, "{:.2f}/μm²")

In [ ]:
fig,axs = plt.subplots(ncols=3, nrows=1,figsize=(15,4))

axs[0].imshow(flat)
axs[0].set_title("Flattened along z")

axs[1].imshow(filled)
axs[1].set_title("Closed")

axs[2].imshow(skel)
axs[2].set_title("Skeletonized")

## 4.Estimate the shape of the nucleus and cell wall
Strategy
* Choose an average volume and aspect ratio
* Convert the CET membrane lattice sites to $(x,y,z)$ coordinates
* Fit ellipsoid to coordinates, using the volume and aspect ratio as extra terms in the loss function

In [12]:
def spheroidLoss(params, coords, membraneVol, targetAspect):
    invRs =  np.array([1/x for x in params[:3]])
    alpha, beta, gamma = params[3:6]
    x0 = params[6:9]
    
    ca, sa = np.cos(alpha), np.sin(alpha)
    cb, sb = np.cos(beta), np.sin(beta)
    cg, sg = np.cos(gamma), np.sin(gamma)
    rotZ0 = np.array([ [ca,-sa,0],[sa,ca,0], [0,0,1] ])
    rotX= np.array([[1,0,0,], [0,cb,-sb],[0,sb,cb] ])
    rotZ1 =np.array( [ [cg,-sg,0],[sg,cg,0], [0,0,1] ])
    rot = rotZ1 @ rotX @ rotZ0
    
    x1 = np.einsum("i,ij,mj->mi",  invRs, rot, coords -x0)
    
    vol = 4*np.pi*reduce(lambda x,y:x*y, params[:3])/3
    aspect =  max(params[:3]) / min(params[:3])
    
    volP =  (vol/membraneVol-1)**2
    aspectP =  (aspect/targetAspect - 1)**2
    
    return np.sum((np.sum(x1**2, axis=1)-1)**2) + volP + aspectP

### 4.1Nucleus

In [ ]:
nucleusVol = nucleusFraction*cellVolume * (1e-6)**3 /latticeSpacing**3
cetNucEnvCoords = np.argwhere(cetNucEnvReg)
chullCoords = \
           cetNucEnvCoords[spspat.ConvexHull(cetNucEnvCoords).vertices]

radius = 0.5*max(np.linalg.norm(x-y) 
                    for x,y in itertools.product(chullCoords,repeat=2))
centroid = np.mean(chullCoords,axis=0)
# The centroid is the arithmetic mean position of all the points in a shape. 
# In this line, np.mean(chullCoords, axis=0) calculates the centroid of the 
# convex hull by averaging the coordinates of its vertices. 
# centroid is the geometical center of nucleus in lattice
params0 = np.array([radius, radius, radius, 
                    0, 0, 0, 
                    centroid[0],  centroid[1],  centroid[2] ])
# basically, scipy optimize minimize the spheroiLoss by finding the optimum value 
# of params0, save back to m.x

m = spopt.minimize(lambda x: spheroidLoss(x, cetNucEnvCoords, 
                                          nucleusVol, nucleusAspect),
                   params0, options=dict(maxiter=100000))
nucParams = m.x
report("nucParas shape",nucParams.shape)
print("nucParams", nucParams[:3],nucParams[6:9])
report("nucleus a-axis", nucParams[0]*latticeSpacing*1e6, "{:.3f} μm")
report("nucleus b-axis", nucParams[1]*latticeSpacing*1e6, "{:.3f} μm")
report("nucleus c-axis", nucParams[2]*latticeSpacing*1e6, "{:.3f} μm")

### 4.2Plasma membrane

In [14]:
membraneVol = (1-cellWallFraction)*cellVolume * (1e-6)**3 \
                       /latticeSpacing**3 # remaining 25% is cellwall
cetMemCoords = np.argwhere(cetMemReg)
radius = (3*membraneVol/np.pi/4)**0.3333
# report("radius",radius*latticeSpacing*1e6, "{:.3f} μm")
params0 = np.array([radius, radius, radius, 
                    0, 0, 0, 
                    nucParams[6],  nucParams[7],  nucParams[8]])
m = spopt.minimize(lambda x: spheroidLoss(x, cetMemCoords, membraneVol, 
                                          cellAspect), 
                   params0, options=dict(maxiter=100000))
memParams = m.x


In [ ]:
report("memParams shape",memParams.shape)
print("memParams", memParams[:3],memParams[6:9])
report("membrane a-axis", memParams[0]*latticeSpacing*1e6, "{:.3f} μm")
report("membrane b-axis", memParams[1]*latticeSpacing*1e6, "{:.3f} μm")
report("membrane c-axis", memParams[2]*latticeSpacing*1e6, "{:.3f} μm")

first 3 paras are the optimized dimensions of membrane, last three paras are the optimized centroid of nucleus. 

middle 3 are rotational angles

## 4.3Cell wall

Compute thickness of cell wall based on its volume fraction

In [ ]:
def volDiff(delta, wallCellRatio):
    return (1 - reduce(lambda x,y: x*y, (r/(r+delta) 
                                          for r in memParams[:3])) 
            - wallCellRatio )
d = spopt.brentq(lambda x: volDiff(x, cellWallFraction), 
                 0, max(memParams[:3]))
cellWallParams = memParams.copy()
cellWallParams[:3] += d
report("cell wall thickness", d*latticeSpacing*1e9, "{:.1f} nm")

### 4.4 Now to generate the lattice


First need to determine the dimensions of the lattice and translation between the new and working lattice

In [ ]:
def lmDim(x): 
    return 2*int(32*np.ceil(x/32))
# first 3 para of memParams are cell sizes xyz; 
n = max(map(lmDim, memParams[:3]))

report("dimension of model n:",n)
Bfull = RegionBuilder(dims=(n,n,n))
print('x direction center of membrane after optimization: ', memParams[6])
# try solve the problem of excedd the 192 boundary by making memParas[6]= 0,
# since it is now negative value, x dir of centroid of membrane center


tx = Bfull.center - memParams[6:9]
# print(Bfull.center, memParams[6:9])
# print('tx:   ',tx)

memParamsTx = memParams.copy()
memParamsTx[6:9] += tx
print('memParamsTx:   ',memParamsTx)
nucParamsTx = nucParams.copy()
nucParamsTx[6:9] += tx
print(nucParamsTx)
print('nucParamsTx:   ',nucParamsTx)
cellWallParamsTx = cellWallParams.copy()
cellWallParamsTx[6:9] += tx
print('cellWallParamsTx:   ',cellWallParamsTx[6:9])

x0,y0,z0 = int(tx[0]), int(tx[1]), int(tx[2])

x1 = x0+cetErReg.shape[0]
y1 = y0+cetErReg.shape[1]
z1 = z0+cetErReg.shape[2]
print("center for implement cet: ", x0,y0,z0)


### 4.5Copy into new lattice

In [ ]:
cetErFullReg = Bfull.emptyLatticeMask()
print(cetErFullReg[x0:x1,:,:].shape)
cetErFullReg[x0:x1, y0:y1, z0:z1] = cetErReg[0:90,:,:]
cetMitoFullReg = Bfull.emptyLatticeMask()
cetMitoFullReg[x0:x1, y0:y1, z0:z1] =  cetMitoReg[0:90,:,:]
cetMemFullReg = Bfull.emptyLatticeMask()
cetMemFullReg[x0:x1, y0:y1, z0:z1] =  cetMemReg[0:90,:,:]
cetNucEnvFullReg = Bfull.emptyLatticeMask()
cetNucEnvFullReg[x0:x1, y0:y1, z0:z1] =  cetNucEnvReg[0:90,:,:]

###  4.6Build the nucleus and plasma membrane

In [19]:
def ellipseFn(builder, params, thickness=3):
    radius = params[:3]
    angles = params[3:6]
    center = params[6:9]
    dr = thickness/2
    radius = np.maximum(np.zeros(3), radius - dr)
    if thickness == 0:
        e = builder.ellipsoid(radius=params[:3], 
                              angles=angles, center=center)
        return e & ~builder.erode(e,1)
    else:
        return ( builder.ellipsoid(radius=radius+dr, 
                                   angles=angles, center=center)
                  & ~builder.ellipsoid(radius=radius-dr, 
                                       angles=angles, center=center))

membrane visualiztion

In [ ]:
# args dont match with the current & version in this dir showBinaryLattices
# def showBinaryLattices(binLattices, mode="widget"):
fitMemFullReg=ellipseFn(Bfull, memParamsTx, thickness=0)
# RegionBuilder.showBinaryLattices(fit=fitMemFullReg, cet=cetMemFullReg)
RegionBuilder.showBinaryLattices({"fit":fitMemFullReg,"cet": cetMemFullReg })
print(Bfull.shape)

nucleus visualization

In [ ]:
fitNucEnvFullReg=ellipseFn(Bfull, nucParamsTx, 
                           thickness=envelopeThickness/latticeSpacing)
RegionBuilder.showBinaryLattices({"fit":fitNucEnvFullReg, "cet":cetNucEnvFullReg})

### 4.7 Build the cell wall

In [22]:
entireCell = Bfull.convexHull(ellipseFn(Bfull, cellWallParamsTx, 
                                        thickness=0))
fitCellWallFullReg = entireCell & ~ Bfull.convexHull(fitMemFullReg)

### 4.8 Add nuclear pores
Calculate the surface area of the nucleus and use the previously computed experimental pore density to choose the number of pores.

In [ ]:
def ellipsoidSA(a,b,c):
    a,b,c = reversed(sorted([a,b,c]))
    phi = np.arccos(c/a)
    k = np.sqrt(a**2*(b**2-c**2)/b**2/(a**2-c**2))
    return  (2*np.pi*c**2  
              + 2*np.pi*a*b/np.sin(phi) 
                *(sps.ellipeinc(phi,k)*np.sin(phi)**2 
                    + sps.ellipkinc(phi, k)*np.cos(phi)**2))

nPores = round(poreDensity
               *ellipsoidSA( *(latticeSpacing*nucParams[:3]) ))

report("Number of nuclear pores", int(nPores))

#### 4.8.1 Place pores in the nuclear envelope

In [24]:
nucFilled = B.convexHull(fitNucEnvFullReg)
poreRadius = int(nucPoreRadius/latticeSpacing)

# do not allow pores to be closer than 3 radii to each other
# this prevents pores from combining into a single pore with 
# the wrong # radius
exclusionRadius = 3*poreRadius

#### 4.8.2 First choose the pore locations

We generate the nuclear envelope and pores at the same time

In [25]:
poreSurface = B.dilate(~nucFilled, 1) & nucFilled

Pore locations are chosen at random from the surface of the nucleus.
Each time a location is chosen a ball of radius $r_{pore}+r_{exc}$ is
placed at the new location in an exclusion lattice, which is tested 
when new locations are sampled.

In [26]:
exclusion = np.zeros_like(poreSurface)
exclusionBall = B.sphereStructElem(poreRadius+exclusionRadius)
r = exclusionBall.shape[0]//2

poreLoc = []
for ct in range(int(nPores)):
    pos = np.argwhere(poreSurface & ~exclusion)
    loc = pos[np.random.choice(np.arange(pos.shape[0]))]
    x,y,z = loc
    poreLoc.append(loc)
    exclusion[x-r:x+r+1, y-r:y+r+1, z-r:z+r+1] |= exclusionBall

#### 4.8.3 Then subtract from the nuclear shell

We produce a mask lattice where a cylinder defining the pore is placed at each location and rotated to follow the radius vector. This is subtracted from the nuclear shell to form the nuclear envelope. The intersection of the nuclear shell and this mask forms the nuclear pore region.

In [27]:
poresMask = np.zeros_like(poreSurface)
cylLen = max(nucParamsTx[:3])

for ct,loc in enumerate(poreLoc):
    dv = loc - nucParamsTx[6:]
    normal = dv/np.sqrt(dv@dv)
    alpha = np.arctan2(normal[0], normal[1])
    beta = np.arccos(normal[2])
    gamma = 0
    poresMask |= Bfull.cylinder(poreRadius, cylLen, 
                                angles=[alpha,beta,gamma], 
                                center=cylLen*normal + nucParamsTx[6:])
    

nucenvFit = fitNucEnvFullReg & ~poresMask
fitNucPoresFullReg = fitNucEnvFullReg & ~nucenvFit
fitNucEnvFullReg = nucenvFit

In [ ]:
RegionBuilder.showBinaryLattices({"mask":poresMask, "pores":fitNucPoresFullReg, \
    "envelope": fitNucEnvFullReg})

## 5. Resize the lattice again to crop off empty regions

In [29]:
def dimfn(u):
    v = (u+1)%3
    w = (u+2)%3
    ct = np.sum(entireCell, axis=(v,w))
    u0 = np.where(ct>0)[0][0] 
    uw = np.sum(ct>0)
    uw32 = int(32*np.ceil(uw/32))
    spc = uw32-uw
    pad0 = spc//2
    return u0, uw, pad0, uw32

x0, xw, px0, nx = dimfn(0)
y0, yw, py0, ny = dimfn(1)
z0, zw, pz0, nz = dimfn(2)

def resize(l0):
    l1 = np.zeros((nx,ny,nz), dtype=np.bool)
    l1[px0:px0+xw,py0:py0+yw,pz0:pz0+zw] = \
                                      l0[x0:x0+xw, y0:y0+yw, z0:z0+zw]
    return l0

In [ ]:
cetErCropReg = resize(cetErFullReg)
cetMemCropReg = resize(cetMemFullReg)
cetMitoCropReg = resize(cetMitoFullReg)
cetNucEnvCropReg = resize(cetNucEnvFullReg)
fitCellWallCropReg = resize(fitCellWallFullReg)
fitMemCropReg = resize(fitMemFullReg)
fitNucEnvCropReg = resize(fitNucEnvFullReg)
fitMemCropReg = resize(fitMemFullReg)
fitNucPoresCropReg = resize(fitNucPoresFullReg)

## 6 Generate the nucleoplasm and cytoplasm

In [31]:
entireCell = B.convexHull(fitCellWallCropReg)

fitNucleoplasmCropReg = (
    B.convexHull(fitNucEnvCropReg|fitNucPoresCropReg)  
      & ~(fitNucEnvCropReg|fitNucPoresCropReg))

fitCytoplasmCropReg = entireCell & ~( fitCellWallCropReg 
                                     | fitMemCropReg 
                                     | fitNucEnvCropReg 
                                     | fitMemCropReg 
                                     | fitNucPoresCropReg 
                                     | fitNucleoplasmCropReg)

## 7. Add the vacuole

In [32]:
B = RegionBuilder(dims=fitCytoplasmCropReg.shape)
vacRadius = (3*vacuoleFraction*cellVolume/4/np.pi)**0.333 \
                *1e-6/latticeSpacing

We create a lattice of possible vacuole centers by eroding the cytoplasm region lattice by the maximum vacuole radius. The vacuole is an ellipsoid with random radii and orientations. If the random vacuole fits in the cytoplasm, it is accepted.

In [33]:
mask = fitCytoplasmCropReg.copy()
for i in range(int(vacRadius)):
    mask = B.erode(mask, 1)
poss = np.argwhere(mask)
while True:
    sa = 0.1*np.random.random() + 0.9
    sb = 0.1*np.random.random() + 0.9
    sc = 1/sa/sb
    radius = np.array([sa*vacRadius, sb*vacRadius, sc*vacRadius])
    # do modification to only accept vacuole outlayer
    angle = np.random.random(3)*2*np.pi
    loc = poss[np.random.randint(poss.shape[0]),:]
    obj = B.ellipsoid(radius=radius, angles=angle, center=loc)
    
    obj_dilate = B.dilate(obj, se = B.se26)
    obj_layer = obj_dilate & ~ obj
    s = np.sum(obj_dilate&~fitCytoplasmCropReg)
    if s == 0:
        break

fitVacuoleCropReg = obj_layer
fitCytoplasmCropReg &= ~ obj_dilate

## 8.Add mitochondria

Same idea as the vacuole, however we are now placing $n$ mitochondria. The lattice of prospective locations is updated each as each new object is added.

In [34]:
mitoVols = 0.2*np.random.random(nMito)+0.8
mitoVols /= np.sum(mitoVols)
mitoAspectL = 0.5*np.random.random(nMito) + 2
mitoAspectS = np.sqrt(1/mitoAspectL)
mitoRadius = [(3*f*mitoFraction*cellVolume/4/np.pi)**0.333
                 *1e-6/latticeSpacing for f in mitoVols]
mitoRadii = np.array([[l*r, s*r, s*r]
                for l,s,r in zip(mitoAspectL, mitoAspectS, mitoRadius)])
fitMitoCropReg = None

for radius in mitoRadii:
    poss = np.argwhere(fitCytoplasmCropReg)
    while True:
        angle = np.random.random(3)*2*np.pi
        loc = poss[np.random.randint(poss.shape[0]),:]
        obj = B.ellipsoid(radius=radius, angles=angle, center=loc)
        obj_dilate = B.dilate(obj, se=B.se26)
        # obj_layer = obj &~ B.ellipsoid(radius=radius-1, angles=angle, center=loc)
        obj_layer = obj_dilate &~ obj
        s = np.sum(obj_dilate&~fitCytoplasmCropReg)
        if s == 0:
            break
    # only get layer of mito
    
    if fitMitoCropReg is None:
        fitMitoCropReg = obj_layer
    else:
        fitMitoCropReg |= obj_layer
        
    fitCytoplasmCropReg &= ~obj_dilate

## 9.Ribosomes

We place ribosomes randomly throughout the cytoplasm, while ensuring that the ribosomes do not cluster together. 

In [35]:
%%cython
# cython: boundscheck=False
# cython: cdivision=True
# cython: wraparound=False

from libc.stdlib cimport *
from posix.stdlib cimport *
from libc.stdint cimport *

cdef int64_t randIntLessThan(int64_t n):
    cdef int64_t x = random()
    #This loop generates random numbers until it finds one that is 
    # within a range that can be evenly divided by n.
    while x >= RAND_MAX - (RAND_MAX % n):
        x = random()
    return x % n

def placeRibosomes(int n, int64_t[:,:,:] avail, int64_t[:,:,:] rib):
    '''
    n : # of ribosomes
    avail: available position to put ribosomes
    rib: the 3d xyz coord of placed rib
    '''
    cdef int64_t nx, ny, nz, i, j, k
    nx,ny,nz = avail.shape[:3]
    
    while n > 0:
        i = 1+randIntLessThan(nx-2)
        j = 1+randIntLessThan(ny-2)
        k = 1+randIntLessThan(nz-2)
        if avail[i,j,k]:
            avail[i-1:i+2,j,k] = 0
            avail[i,j-1:j+2,k] = 0
            avail[i,j,k-1:k+2] = 0
            rib[i,j,k] = 1
            # ribfull[i,j,k] = 1
            # ribfull[i-1:i+2,j,k] = 1
            # ribfull[i,j-1:j+2,k] = 1
            # ribfull[i,j,k-1:k+2] = 1
            n -= 1

In [36]:
avail = B.erode(fitCytoplasmCropReg,1).astype(np.int64)
ribTmp = np.zeros(avail.shape, dtype=np.int64)
placeRibosomes(nRibosomes, avail, ribTmp)

fitRibCropReg = np.array(ribTmp, dtype=bool)
# remove the region of ribosomes from cytoplasm
fitCytoplasmCropReg &= ~fitRibCropReg


### 9.2Alternative: Place ribosomes based on 1/r law from Nucleus center

Generate bin for ribosomes nums 

In [37]:
# # avail = B.erode(fitCytoplasmCropReg,1).astype(np.int64) # transform it into a int64 array

# ribTmp = np.zeros(fitCytoplasmCropReg.shape, dtype=np.int64)  # dot, only middle one
# ribFull = np.zeros(fitCytoplasmCropReg.shape, dtype=np.int64) # full shape, 7 cubes
# # get 1% of the original 195,000, for education minecarft purpose less density
# #https://bionumbers.hms.harvard.edu/files/Comparison%20of%20ribosome%20number%20and%20density%20between%20M.%20tuberculosis%20and%20yeast%20cells.pdf
# nRibosomes = 1950
# # placeRibosomes(nRibosomes, avail, ribTmp, ribFull)

# # 2. distribution of ribosomes
# env_coord = np.argwhere(fitNucEnvCropReg)
# mem_coord = np.argwhere(fitMemCropReg)
# from scipy.spatial import distance_matrix
# from math import ceil
# dis_max = np.max(np.min(distance_matrix(mem_coord,env_coord),axis=1))
# dis_max = ceil(dis_max)
# print('dis_max',dis_max)
# # generate a distribtion bin 
# ribo_bin = np.zeros(int(dis_max)+1)
# #ribosome density is 1/x
# for i in range(1, int(dis_max)+1):
#     ribo_bin[i] = 1/i
# #normalized the bin
# ribo_bin = ribo_bin/np.sum(ribo_bin)
# #get the ribo number in each bin
# for i in range(1, int(dis_max)+1):
#     ribo_bin[i] = round(ribo_bin[i]*nRibosomes)

# print('ribo_bin',ribo_bin)
# print('ribo_bin',np.sum(ribo_bin))

In [38]:
# # generate availabe layers based on the distance to the nucleus

# for i in range(1, int(dis_max)+1):
#     layerParamsTx = np.copy(nucParamsTx)
#     layerParamsTx[:3] += i
#     entireEnv = Bfull.convexHull(ellipseFn(Bfull, layerParamsTx, 
#                                             thickness=0))
#     fitLayerFull = entireEnv & ~ Bfull.convexHull(fitNucEnvFullReg|fitNucPoresFullReg)
#     '''entireCell = Bfull.convexHull(ellipseFn(Bfull, cellWallParamsTx, 
#                                             thickness=0))
#     fitCellWallFullReg = entireCell & ~ Bfull.convexHull(fitMemFullReg)'''
#     # chop off empty place
    
#     fitLayerCrop = resize(fitLayerFull)
#     # print('fitLayerCrop',fitLayerCrop.shape,np.sum(fitLayerCrop),fitLayerCrop.dtype)
    
#     fitLayerCropAvail = fitLayerCrop & fitCytoplasmCropReg
    
#     # RegionBuilder.showBinaryLattices({"layer":fitLayerCropAvail, "nuc":fitNucEnvCropReg})
#     # avail = B.erode(fitLayerCropAvail,1).astype(np.int64)
#     avail = fitLayerCropAvail.astype(np.int64)
#     n = ribo_bin[i]
   
#     placeRibosomes(n, avail, ribTmp, ribFull)
#     if i % 100 ==0:
#         print("finish: {}".format(i/(int(dis_max)+1)))


In [39]:
# import numpy as np
# fitRibCropReg =np.array(ribTmp, dtype=bool)
# #this part of the code is to exclude the ribosome from the cytoplasma
# # fitCytoplasmCropReg &= ~fitRibCropReg

# # fitCytoplasmCropReg &= ~fitRibFullCropReg

In [ ]:
# print(fitRibCropReg.shape,np.count_nonzero(fitRibCropReg))
RegionBuilder.showBinaryLattices({"fit":fitRibCropReg})

make the dot nonzero ribosome location to get a full 3d locations transfer. and save the ribisomes location as a `.npy` file.

## 10.We're done! Write the lattice to disk

In [ ]:
latticeData = dict(cetEr=cetErCropReg,
                   cetMem=cetMemCropReg,
                   cetMito=cetMitoCropReg,
                   cetNucEnv=cetNucEnvCropReg,
                   fitCellWall=fitCellWallCropReg,
                   fitVacuole=fitVacuoleCropReg,
                   fitMito=fitMitoCropReg,
                   fitNucEnv=fitNucEnvCropReg,
                   fitMem=fitMemCropReg,
                   fitRib=fitRibCropReg,
                   fitNucPores=fitNucPoresCropReg)
RegionBuilder.showBinaryLattices(latticeData)

In [42]:
# Define the IDs for each region
ids = dict()
lattice = np.zeros(cetErCropReg.shape, dtype=np.uint8)

# Assign IDs to each region type
ids['cetEr'] = 1
ids['cetMem'] = 2
ids['cetMito'] = 3
ids['cetNucEnv'] = 4
ids['fitCellWall'] = 5
ids['fitVacuole'] = 6
ids['fitMito'] = 7
ids['fitNucEnv'] = 8
ids['fitMem'] = 9
ids['fitRib'] = 10
ids['fitNucPores'] = 11
ids['fitNucleoplasm'] = 12
ids['fitCytoplasm'] = 13

# Populate the lattice with the corresponding IDs
lattice[cetErCropReg] = ids['cetEr']
lattice[cetMemCropReg] = ids['cetMem'] 
lattice[cetMitoCropReg] = ids['cetMito']
lattice[cetNucEnvCropReg] = ids['cetNucEnv']
lattice[fitCellWallCropReg] = ids['fitCellWall']
lattice[fitVacuoleCropReg] = ids['fitVacuole']
lattice[fitMitoCropReg] = ids['fitMito']
lattice[fitNucEnvCropReg] = ids['fitNucEnv']
lattice[fitMemCropReg] = ids['fitMem']
lattice[fitRibCropReg] = ids['fitRib']
lattice[fitNucPoresCropReg] = ids['fitNucPores']
lattice[fitNucleoplasmCropReg] = ids['fitNucleoplasm']
lattice[fitCytoplasmCropReg] = ids['fitCytoplasm']

# Save the data using pickle
with lzma.open(f"yeastLattice.{decimation}.pkl.xz", "wb") as f:
    pickle.dump(
        dict(lattice=lattice,
             ids=ids,
             latticeSpacing=latticeSpacing,
             decimation=decimation),
        f
    )

This is the end.
